In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%run ../tasking_manager_stats/data_management
%run ../tasking_manager_stats/map_tools

# Validation spent time one project 

In [ ]:
project_id = 5861

In [ ]:
db = Database(project_id)

In [ ]:
start = db.get_start_date()
start

In [ ]:
nb_days = compute_nb_days(db, start)
nb_days

In [ ]:
validation_day_duration = np.zeros(nb_days + 1)
validation_cumulative_day_task = np.zeros(nb_days + 1)
unvalidated_tasks = []
mapping_day_duration = np.zeros(nb_days + 1)
mapping_cumulative_day_task = np.zeros(nb_days + 1)
unmapped_tasks = []

In [ ]:
def add_task_spent_time(task_data, day_duration, task_type_lock):
    for task_hist in task_data['taskHistory']:
        if task_hist['action'] == task_type_lock:
            date = pd.to_datetime(task_hist['actionDate']).date()
            day = (date - start).days

            date_duration = pd.to_datetime(task_hist['actionText'])
            duration = date_duration.second + 60 * date_duration.minute + 3600 * date_duration.hour

            day_duration[day] += duration

In [ ]:
def add_task_validation_spent_time(task_data, day_duration):
    add_task_spent_time(task_data, day_duration, 'LOCKED_FOR_VALIDATION')

In [ ]:
def add_task_mapping_spent_time(task_data, day_duration):
    add_task_spent_time(task_data, day_duration, 'LOCKED_FOR_MAPPING')

In [ ]:
def add_task_validation_status(db, task_id, cumulative_day_task, unvalidated_tasks):
    task_validated = np.zeros(nb_days + 1)
    for task_hist in reversed(db.get_task_history()[str(task_id)]['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate']).date()
        day = (date - start).days
        if task_hist['actionText'] == 'VALIDATED':
            task_validated[day:] = 1
    cumulative_day_task += task_validated
    
    if task_validated[-1] == 0:
        unvalidated_tasks.append(task_id) 

In [ ]:
def add_task_mapping_status(db, task_id, cumulative_day_task, unmapped_tasks):
    task_mapped = np.zeros(nb_days + 1)
    for task_hist in reversed(db.get_task_history()[str(task_id)]['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate']).date()
        day = (date - start).days
        if task_hist['actionText'] == 'MAPPED':
            task_mapped[day:] = 1
        elif task_hist['actionText'] == 'INVALIDATED':
            task_mapped[day:] = 0
    cumulative_day_task += task_mapped
    
    if task_mapped[-1] == 0:
        unmapped_tasks.append(task_id) 

In [ ]:
for task_id in db.get_task_ids():
    add_task_validation_spent_time(db.get_task_history()[str(task_id)], validation_day_duration)
    add_task_validation_status(db, task_id, validation_cumulative_day_task, unvalidated_tasks)
    add_task_mapping_spent_time(db.get_task_history()[str(task_id)], mapping_day_duration)
    add_task_mapping_status(db, task_id, mapping_cumulative_day_task, unmapped_tasks)

In [ ]:
validation_day_duration

In [ ]:
validation_cumulative_day_task

In [ ]:
unvalidated_tasks

In [ ]:
mapping_day_duration

In [ ]:
mapping_cumulative_day_task

In [ ]:
validation_day_duration.sum() / 3600.0

In [ ]:
times = []
for day in range(nb_days + 1):
    times.append(start + pd.Timedelta(days=day))

In [ ]:
dpi = 50

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, validation_day_duration / 3600.0, linewidth=2)
ax.set_title('Validation spent time in #' + str(project_id) + f' (total = {validation_day_duration.sum() / 3600.0:.3} h)', fontsize=16)
ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Time spent in hours', fontsize=16)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_validation_spent_time', dpi=dpi)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, validation_day_duration.cumsum() / 3600.0, linewidth=2)
ax.set_title('Cumulative validation spent time in #' + str(project_id) + f' (total = {validation_day_duration.sum() / 3600.0:.3} h)', fontsize=16)
ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Time spent in hours', fontsize=16)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_cumulative_validation_spent_time', dpi=dpi)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, validation_day_duration.cumsum() / 3600.0, linewidth=2)

ax2 = ax.twinx()
color = 'tab:green'
ax2.set_ylabel('Number of validated tasks', color=color)
ax2.plot(times, validation_cumulative_day_task, color=color)
ax2.tick_params(axis='y', labelcolor=color)

title = 'Cumulative validation spent time in #' + str(project_id) + f' (total = {validation_day_duration.sum() / 3600.0:.3} h)'
title = title + f'\n{int(validation_cumulative_day_task[-1])} validated tasks on {len(db.get_task_ids())}'
ax.set_title(title)
ax.set_xlabel('Date')
ax.set_ylabel('Time spent in hours')
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, ax2.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels() +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_cumulative_validation_spent_time_and_validated_tasks', dpi=dpi)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, mapping_day_duration.cumsum() / 3600.0, linewidth=2)

ax2 = ax.twinx()
color = 'tab:green'
ax2.set_ylabel('Number of mapped tasks', color=color)
ax2.plot(times, mapping_cumulative_day_task, color=color)
ax2.tick_params(axis='y', labelcolor=color)

title = 'Cumulative mapping spent time in #' + str(project_id) + f' (total = {mapping_day_duration.sum() / 3600.0:.4} h)'
title = title + f'\n{int(mapping_cumulative_day_task[-1])} mapped tasks on {len(db.get_task_ids())}'
ax.set_title(title)
ax.set_xlabel('Date')
ax.set_ylabel('Time spent in hours')
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, ax2.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels() +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_cumulative_mapping_spent_time_and_validated_tasks', dpi=dpi)

# All projects

In [ ]:
import datetime
start = datetime.datetime(2019, 1, 1).date()
start

In [ ]:
nb_days = (datetime.datetime.now().date() - start).days
nb_days

In [ ]:
def add_task_mapping_and_validation_status(db, task_id, cumulative_day_mapped,
                                           cumulative_day_validated, unvalidated_tasks):
    task_mapped = np.zeros(nb_days + 1)
    task_validated = np.zeros(nb_days + 1)
    for task_hist in reversed(db.get_task_history()[str(task_id)]['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate']).date()
        day = (date - start).days
        if day < 0:
            continue
        if task_hist['actionText'] == 'VALIDATED':
            task_validated[day:] = 1
        if task_hist['actionText'] == 'MAPPED':
            task_mapped[day:] = 1
    cumulative_day_mapped += task_mapped
    cumulative_day_validated += task_validated
    
    if task_validated.max() == 0:
        unvalidated_tasks.append(task_id) 

In [ ]:
day_duration = np.zeros(nb_days + 1)
cumulative_day_mapped = np.zeros(nb_days + 1)
cumulative_day_validated = np.zeros(nb_days + 1)
unvalidated_tasks = []

In [ ]:
projects = ['5847', '5504', '5889', '5654', '4438', '4388']

In [ ]:
for project_id in projects:
    db = Database(project_id)
    for task_id in db.get_task_ids():
        add_task_validation_spent_time(db.get_task_history()[str(task_id)], day_duration)
        add_task_validation_status(db, task_id, cumulative_day_task, unvalidated_tasks)
    time.sleep(10)

# Project times

In [ ]:
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
def get_mapping_validation_dates(db, nb_days):
    min_mapped = nb_days
    max_mapped = 0
    min_validation = nb_days
    max_validation = 0
    for task_id in db.get_task_ids():
        for task_hist in reversed(db.get_task_history()[str(task_id)]['taskHistory']):
            date = pd.to_datetime(task_hist['actionDate']).date()
            day = (date - start).days
            if task_hist['actionText'] == 'VALIDATED':
                min_validation = min(min_validation, day)
                max_validation = max(max_validation, day)
            elif task_hist['actionText'] == 'MAPPED':
                min_mapped = min(min_mapped, day)
                max_mapped = max(max_mapped, day)
    return min_mapped, max_mapped, min_validation, max_validation

In [ ]:
min_mapped, max_mapped, min_validation, max_validation

In [ ]:
min_mapped, max_mapped, min_validation, max_validation = get_mapping_validation_dates(db, nb_days)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot([start, start + pd.Timedelta(days=nb_days)], [0, 0], linewidth=30, color='black')
ax.plot([start + pd.Timedelta(days=min_mapped), start + pd.Timedelta(days=max_mapped)], [0-0.15, 0-0.15],
        linewidth=30, color=(254/255., 231/255., 156/255.))
ax.plot([start + pd.Timedelta(days=min_validation), start + pd.Timedelta(days=max_validation)], [0-0.3, 0-0.3],
        linewidth=30, color=(152/255., 203/255., 151/255.))
ax.plot([start, start + pd.Timedelta(days=nb_days)], [-1, -1], linewidth=30, color='black')
ax.text(start, 0+0.15, db.get_project_name(), fontsize=16)
ax.text(start, -1+0.15, db.get_project_name(), fontsize=16)
ax.set_xlabel('Date', fontsize=16)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
ax.get_yaxis().set_visible(False)

In [ ]:
projects = ['5504', '5848', '5889', '5654', '5861']

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
i = 0
for project_id in projects:
    db = Database(project_id)
    start = db.get_start_date()
    try:
        nb_days = compute_nb_days(db, start)
        
        ax.plot([start, start + pd.Timedelta(days=nb_days)], [i, i], linewidth=30, color='black')
        ax.text(start - pd.Timedelta(days=15), i + 0.15, db.get_project_name(), fontsize=16)
        
        min_mapped, max_mapped, min_validation, max_validation = get_mapping_validation_dates(db, nb_days)
        
        ax.plot([start + pd.Timedelta(days=min_mapped), start + pd.Timedelta(days=max_mapped)], [i - 0.15, i - 0.15],
        linewidth=30, color=(254/255., 231/255., 156/255.))
        ax.text(start + pd.Timedelta(days=min_mapped), i - 0.15, f'Mapping ({max_mapped - min_mapped} days)', fontsize=16)
        
        ax.plot([start + pd.Timedelta(days=min_validation), start + pd.Timedelta(days=max_validation)], [i - 0.3, i - 0.3],
        linewidth=30, color=(152/255., 203/255., 151/255.))
        ax.text(start + pd.Timedelta(days=min_validation), i - 0.3,
                f'Validation ({max_validation - min_validation} days)', fontsize=16)

        ax.set_xlabel('Date', fontsize=16)
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(16)
        i += 1
    except:
        print(f'Error with project_id={project_id}')
        pass
ax.get_yaxis().set_visible(False)
plt.savefig('../data/projects.png', dpi=dpi)